In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras.utils import plot_model

In [ ]:
dataset = 'data'
model_save_path = 'The saved model/model.keras'

In [ ]:
classes = 4

In [ ]:
def load_dataset(dataset_path):
    x = []
    y = []
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

    # Loop through dataset directory
    for exercise_folder in os.listdir(dataset_path):
        exercise_label = exercise_folder
        exercise_folder_path = os.path.join(dataset_path, exercise_folder)  

        # Loop through video files in exercise folder
        for video_file in os.listdir(exercise_folder_path):
            video_path = os.path.join(exercise_folder_path, video_file)
            cap = cv2.VideoCapture(video_path)
            while cap.isOpened():
                success, image = cap.read()
                if not success:
                    break
                # Process image using MediaPipe Pose Detection
                image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                result = pose.process(image_rgb)
                if result.pose_landmarks:
                    # Extract pose landmarks
                    landmarks = [[lm.x, lm.y] for lm in result.pose_landmarks.landmark]
                    x.append(landmarks)
                    y.append(exercise_label)
            cap.release()
    return np.array(x), np.array(y)

In [ ]:
x, y = load_dataset(dataset)

In [ ]:
print(y)

In [ ]:
# Convert labels to numerical values
label_map = {
    'deadlift_False': 0,
    'deadlift_True': 1,
    'lat_pulldown_False': 2,
    'lat_pulldown_True': 3
}
y = np.array([label_map[label] for label in y])

In [ ]:
print(y)

In [ ]:
if x.dtype != np.float32:
    x = x.astype(np.float32)
if y.dtype != np.float32:
    y = y.astype(np.float32)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
X_train.shape

In [ ]:
from keras.layers import  BatchNormalization
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(X_train.shape[1], X_train.shape[2])),
    
    # Convolutional layers
    tf.keras.layers.Conv1D(64, kernel_size=3, activation='tanh', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    
    tf.keras.layers.Conv1D(128, kernel_size=3, activation='tanh', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    
    tf.keras.layers.Conv1D(256, kernel_size=3, activation='tanh', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    
    # Flatten layer
    tf.keras.layers.Flatten(),
    
    # Dense layers
    tf.keras.layers.Dense(512, activation='tanh'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.BatchNormalization(),
    
    # Output layer
    tf.keras.layers.Dense(classes, activation='softmax')
])

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    model_save_path, verbose=1, save_weights_only=False)
# Callback for early stopping
es_callback = tf.keras.callbacks.EarlyStopping(patience=20, verbose=1)

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history = model.fit(X_train,y_train,epochs=150,batch_size=64,validation_split=0.2,callbacks=[cp_callback, es_callback])


In [ ]:
model.summary()

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)

In [ ]:
# Evaluate model on training data
train_loss, train_accuracy = model.evaluate(X_train, y_train)
print("Training Accuracy:", train_accuracy)

# Evaluate model on test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Testing Accuracy:", test_accuracy)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Predict labels for test data
y_pred = np.argmax(model.predict(X_test), axis=1)
# Error Analysis
misclassifications = np.where(y_test != y_pred)[0]
print("Misclassified Instances:", misclassifications)

# Result Analysis
accuracy = np.mean(y_test == y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

# Confusion Matrix
conf_mat = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# Visualization for Test and Train (Learning curves)
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 1])
plt.legend(loc='upper right')
plt.show()

In [ ]:
model = tf.keras.models.load_model(model_save_path)

In [ ]:
print(model.input_shape)

In [ ]:
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

In [ ]:
video_paths = [
    'deadlift-4.mp4',
    'deadlift_false.mp4',
    'false lat pulldown.mp4',
    'lat pulldown_35.mp4'
]

In [ ]:
for video_path in video_paths:
     
    cap = cv2.VideoCapture(video_path)

    # Check if the video/camera opened successfully
    if not cap.isOpened():
        print("Error: Unable to open video file or camera.")
        exit()

    # Initialize MediaPipe Pose Detection
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

    # Function to resize frame while maintaining aspect ratio
    def resize_frame(frame, width=None, height=None):
        h, w = frame.shape[:2]
        if width is None and height is None:
            return frame
        if width is None:
            ratio = height / h
        else:
            ratio = width / w
        dim = (int(w * ratio), int(h * ratio))
        return cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)

    # Loop through the video frames
    while cap.isOpened():
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            break
    
        # Resize the frame for faster processing (adjust the dimensions as needed)
        resized_frame = resize_frame(frame, width=440)  # Adjust width as needed
    
        # Process frame with MediaPipe Pose Detection
        frame_rgb = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
        result = pose.process(frame_rgb)
    
        # Check if pose landmarks are detected
        if result.pose_landmarks:
            # Extract pose landmarks
            landmarks = [[lm.x, lm.y] for lm in result.pose_landmarks.landmark]

            # Preprocess landmarks (reshape, convert to numpy array, etc.)
            # Example:
            landmarks_array = np.array(landmarks, dtype=np.float32)
            landmarks_array = landmarks_array[np.newaxis, ...]  # Add batch dimension

            # Use the model to make predictions
            predictions = model.predict(landmarks_array)
            predicted_class = "not_an_exercise"
            # Example: Print the predicted class
            min_value = 0.5
            if np.max(predictions) >= min_value:
                predicted_class = ""
                if np.argmax(predictions) == 0:
                    predicted_class = "deadlift_False"
                    m = 0
                elif np.argmax(predictions) == 1:
                    predicted_class = "deadlift_True"
                    m = 1
                elif np.argmax(predictions) == 2:
                    predicted_class = "lat_pulldown_False"
                    m = 0
                elif np.argmax(predictions) == 3:
                    predicted_class = "lat_pulldown_True"
                    m = 1                        
            else:
                predicted_class = "not_an_exercise"  
            if m == 0:     
               cv2.putText(resized_frame, predicted_class, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            elif m == 1:
               cv2.putText(resized_frame, predicted_class, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)  
         


        # Display the resized frame
        cv2.imshow("output", resized_frame)
    
        # Break the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the video capture object and close all windows
    cap.release()
cv2.destroyAllWindows()

In [ ]:
cap = cv2.VideoCapture(0)

# Check if the video/camera opened successfully
if not cap.isOpened():
    print("Error: Unable to open video file or camera.")
    exit()

# Initialize MediaPipe Pose Detection
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Function to resize frame while maintaining aspect ratio
def resize_frame(frame, width=None, height=None):
    h, w = frame.shape[:2]
    if width is None and height is None:
        return frame
    if width is None:
        ratio = height / h
    else:
        ratio = width / w
    dim = (int(w * ratio), int(h * ratio))
    return cv2.resize(frame, dim, interpolation=cv2.INTER_AREA)

# Loop through the video frames
while cap.isOpened():
    # Read a frame from the video
    ret, frame = cap.read()
    if not ret:
        break
    
    # Resize the frame for faster processing (adjust the dimensions as needed)
    resized_frame = resize_frame(frame, width=640)  # Adjust width as needed
    
    # Process frame with MediaPipe Pose Detection
    frame_rgb = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
    result = pose.process(frame_rgb)
    
    # Check if pose landmarks are detected
    if result.pose_landmarks:
        # Extract pose landmarks
        landmarks = [[lm.x, lm.y] for lm in result.pose_landmarks.landmark]

        # Preprocess landmarks (reshape, convert to numpy array, etc.)
        # Example:
        landmarks_array = np.array(landmarks, dtype=np.float32)
        landmarks_array = landmarks_array[np.newaxis, ...]  # Add batch dimension

        # Use the model to make predictions
        predictions = model.predict(landmarks_array)
        predicted_class = "not_an_exercise"
        # Example: Print the predicted class
        min_value = 0.5
        if np.max(predictions) >= min_value:
            predicted_class = ""
            if np.argmax(predictions) == 0:
                predicted_class = "deadlift_False"
                m = 0
            elif np.argmax(predictions) == 1:
                predicted_class = "deadlift_True"
                m = 1
            elif np.argmax(predictions) == 2:
                predicted_class = "lat_pulldown_False"
                m = 0
            elif np.argmax(predictions) == 3:
                predicted_class = "lat_pulldown_True"
                m = 1                        
        else:
            predicted_class = "not_an_exercise"  
        if m == 0:     
          cv2.putText(resized_frame, predicted_class, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        else:
          cv2.putText(resized_frame, predicted_class, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            

    # Display the resized frame
    cv2.imshow("output", resized_frame)
    
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

# Predict labels for test data
y_pred = np.argmax(model.predict(X_test), axis=1)
# Error Analysis
misclassifications = np.where(y_test != y_pred)[0]
print("Misclassified Instances:", misclassifications)

# Result Analysis
accuracy = np.mean(y_test == y_pred)
print("Accuracy:", accuracy)
print(classification_report(y_test, y_pred))

# Confusion Matrix
conf_mat = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.title('Confusion Matrix')
plt.show()

# Visualization for Test and Train (Learning curves)
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, 1])
plt.legend(loc='upper right')
plt.show()